# Problem 6.1

In this problem we are asked to consider a homogeneous mixture of sodium and plutonium.  We will assume the only plutonium isotope is Pu-239 and natural sodium is used.  The Pu-239 fraction is 3 weight percent.  The overall mixture density is 1 g/cc.  

Question: what is the fuel utilization and $k_{\infty}$

In [1]:
import openmc

## Materials

In [2]:
core = openmc.Material(name='core');
core.add_element('Na',0.97,'wo');
core.add_nuclide('Pu239',0.03,'wo');
core.set_density('g/cm3',1.0);

materials = openmc.Materials([core]);
materials.export_to_xml();

## Geometry
We will first model the geometry as a sphere with a reflective boundary

In [3]:
R = 5.0
core_g = openmc.Sphere(r=R,boundary_type='reflective');


core_c = openmc.Cell();
core_c.fill = core;
core_c.region = -core_g;
core_c.temperature = 900;

root_u = openmc.Universe();
root_u.add_cells([core_c]);

geometry = openmc.Geometry(root_u);

geometry.export_to_xml();

## Tallies

In [4]:
cell_filter = openmc.CellFilter(core_c);


nu_tally = openmc.Tally(name='nu');
nu_tally.scores = ['nu-fission']
nu_tally.filters = [cell_filter];

abs_tally = openmc.Tally(name='abs');
abs_tally.scores = ['absorption']
abs_tally.nuclides = ['Pu239','Na23'];

fiss_tally = openmc.Tally(name='fission');
fiss_tally.scores = ['fission'];

capture_tally = openmc.Tally(name='n_gamma');
capture_tally.scores = ['(n,gamma)'];
capture_tally.nuclides = ['Pu239'];

tallies = openmc.Tallies([nu_tally,abs_tally,fiss_tally,capture_tally]);

tallies.export_to_xml();


## Settings


In [5]:
settings = openmc.Settings();
settings.batches = 300;
settings.inactive = 100;
settings.particles = 20000;

bounds = [-R,-R,-R,R,R,R];
uniform_dist = openmc.stats.Box(bounds[:3],bounds[3:],
                                   only_fissionable=True);
settings.source = openmc.source.Source(space=uniform_dist);

settings.temperature['method']='interpolation';

settings.export_to_xml();

In [6]:
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

      178/1    1.55845    1.55191 +/- 0.00103
      179/1    1.55900    1.55200 +/- 0.00102
      180/1    1.52852    1.55171 +/- 0.00105
      181/1    1.55017    1.55169 +/- 0.00103
      182/1    1.54725    1.55163 +/- 0.00102
      183/1    1.56453    1.55179 +/- 0.00102
      184/1    1.54829    1.55175 +/- 0.00101
      185/1    1.54153    1.55163 +/- 0.00101
      186/1    1.56053    1.55173 +/- 0.00100
      187/1    1.56609    1.55189 +/- 0.00100
      188/1    1.56604    1.55205 +/- 0.00100
      189/1    1.56070    1.55215 +/- 0.00100
      190/1    1.54147    1.55203 +/- 0.00099
      191/1    1.54340    1.55194 +/- 0.00099
      192/1    1.55900    1.55202 +/- 0.00098
      193/1    1.54595    1.55195 +/- 0.00097
      194/1    1.55245    1.55196 +/- 0.00096
      195/1    1.54730    1.55191 +/- 0.00095
      196/1    1.57643    1.55216 +/- 0.00098
      197/1    1.54467    1.55208 +/- 0.00097
      198/1    1.54200    1.55198 +/- 0.00096
      199/1    1.56503    1.55211 

In [7]:
!cat tallies.out

 ==========================>     TALLY 1: NU     <==========================

 Cell 1
   Total Material
     Nu-Fission Rate                      1.55252 +/- 0.000758147
 =========================>     TALLY 2: ABS     <==========================

 Pu239
   Absorption Rate                      0.893633 +/- 0.000449094
 Na23
   Absorption Rate                      0.106058 +/- 7.00213e-05
 =======================>     TALLY 3: FISSION     <========================

 Total Material
   Fission Rate                         0.538644 +/- 0.000264222
 =======================>     TALLY 4: N_GAMMA     <========================

 Pu239
   (n,gamma)                            0.354989 +/- 0.000201186


The answer calculated for problem 6.1: f = 0.887; from OpenMC it appears to be a pretty close match where the fraction of source neutrons absorbed in Pu239 is roughly 0.89.  

The eigenvalue caluclated in OpenMC is only about 1.567; this is much lower than the value presented in the textbook which computes out to 2.315.  

The higher $k_{\infty}$ from the text is driven by the "nominal 1-group constants for a fast reactor" presented in Table 6.1 of the text.  In particular the microscopic fission cross section is given as 1.85b while the capture cross section is only 0.26b.  In this OpenMC analysis the capture rate is more than half the fission rate.  This seems to be the biggest difference.

I will try to change the temperature of the evaluations to be more representative of a fast reactor.